<a href="https://colab.research.google.com/github/rornellas/tech-challenge-3-ia4devs/blob/main/fine_tunning_tech_challenge_fase_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -q --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install -q transformers datasets
!pip install -q triton

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 require

# Executando no COLAB utilizando GPU

In [1]:
from unsloth import FastLanguageModel
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

torch.cuda.is_available()

base_location = "/content/drive/MyDrive/IA para devs - fiap/Fase 3/Tech Challenge/20240923/"

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


# Processando arquivo de input

In [2]:
import json
from itertools import islice

# Carregar os dados
with open('/content/drive/MyDrive/IA para devs - fiap/Fase 3/Tech Challenge/trn_first10000.json', 'r', encoding='utf-8') as f:
    rows = [
        {
            'title': entry.get('title', ''),
            'description': entry.get('content', '')
        }
        for entry in (json.loads(line) for line in islice(f, 200))
        if len(entry.get('content', '')) > len(entry.get('title', ''))
    ]

print("Tamanho input: ", rows.__len__())

# Exibir a primeira entrada filtrada, se houver
if rows:
    print(json.dumps(rows[0], indent=4, ensure_ascii=False))
else:
    print("Nenhum registro corresponde ao critério.")


Tamanho input:  95
{
    "title": "Girls Ballet Tutu Neon Pink",
    "description": "High quality 3 layer ballet tutu. 12 inches in length"
}


## Realizando limpeza

In [3]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
import json

# Baixar recursos do NLTK
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('brown')

df = pd.DataFrame(rows)

# 2. Remover duplicatas
df = df.drop_duplicates(subset=['title', 'description'])

# 3. Normalizar o texto
def normalize_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'\s+', ' ', text)
        text = text.strip()
    return text

df['title'] = df['title'].apply(normalize_text)
df['description'] = df['description'].apply(normalize_text)

# 4. Remover pontuação
def remove_punctuation(text):
    if isinstance(text, str):
        text = re.sub(r'[^\w\s]', '', text)
    return text

df['title'] = df['title'].apply(remove_punctuation)
df['description'] = df['description'].apply(remove_punctuation)

# 5. Remover stop words
stop_words = set(stopwords.words('english'))  # Ajuste para 'portuguese' se necessário

def remove_stopwords(text):
    if isinstance(text, str):
        tokens = nltk.word_tokenize(text)
        tokens = [word for word in tokens if word not in stop_words]
        text = ' '.join(tokens)
    return text

df['title'] = df['title'].apply(remove_stopwords)
df['description'] = df['description'].apply(remove_stopwords)

# 6. Lematização
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    if isinstance(text, str):
        tokens = nltk.word_tokenize(text)
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
        text = ' '.join(tokens)
    return text

df['title'] = df['title'].apply(lemmatize_text)
df['description'] = df['description'].apply(lemmatize_text)

# 7. Correção ortográfica (opcional)
def correct_spelling(text):
    if isinstance(text, str):
        blob = TextBlob(text)
        corrected = str(blob.correct())
        return corrected
    return text

# Descomente as linhas abaixo se desejar aplicar correção ortográfica
# df['title'] = df['title'].apply(correct_spelling)
# df['description'] = df['description'].apply(correct_spelling)

# 8. Remover entradas vazias
df = df.dropna(subset=['title', 'description'])
df = df[(df['title'].str.strip() != '') & (df['description'].str.strip() != '')]

# 9. Converter para array JSON atribuído a uma variável
data = df.to_dict(orient='records')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


## Criando Modelo para Treinamento

In [9]:
from unsloth import FastLanguageModel

# Nome do modelo a ser utilizado
model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"
max_seq_length = 4096
dtype = None
load_in_4bit = True

In [17]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.9.post2: Fast Mistral patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [18]:
alpaca_prompt = """### Instruction:
Generate a comprehensive description based on the provided title.

### Title:
{}

### Description:
{}"""

In [19]:
EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    inputs       = examples["title"]
    outputs      = examples["description"]
    texts = []
    for input, output in zip(inputs, outputs):

        text = alpaca_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import Dataset

dataset = Dataset.from_list(data)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/95 [00:00<?, ? examples/s]

In [20]:
# Dividindo o dataset em 80% treino e 20% validação
train_test_split = dataset.train_test_split(test_size=0.2, seed=42)

# Separando em datasets de treino e validação
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

print(train_dataset[0])
print("\n")
print(eval_dataset[0])

{'title': 'diary samuel pepys 1660', 'description': 'lathammatthews transcription pepys diary one glory contemporary english publishing time pleasure pepys reading pleasure experience pepys diary cheerful selfreport man eminent naval history historical witness unobjectionable hedonist guardian one finest feat long history scholarship pepys diary exegi monumentum aere perennius observer editor achieved impossible one read diary perfectly easily month month year year last really learned edition learning put disposal layman new statesman isnt often one encounter publication especially magnitude achieves complete perfection doubt sir arthur bryant', 'text': '### Instruction:\nGenerate a comprehensive description based on the provided title.\n\n### Title:\ndiary samuel pepys 1660\n\n### Description:\nlathammatthews transcription pepys diary one glory contemporary english publishing time pleasure pepys reading pleasure experience pepys diary cheerful selfreport man eminent naval history hist

In [21]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.9.post2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Executando antes do treino

In [22]:
#{
#    "instruction": "Summarize this content",
#    "input": "Girls Ballet Tutu Neon Pink",
#    "output": "High quality 3 layer ballet tutu. 12 inches in length"
#}

# Executando pergunta antes do treino
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        train_dataset[0]['title'], # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<s> ### Instruction:
Generate a comprehensive description based on the provided title.

### Title:
diary samuel pepys 1660

### Description:
The Diary of Samuel Pepys, written in 1660, is a primary source detailing the life of Samuel Pepys, a prominent English statesman and official who served as a Member of Parliament and held various positions in the navy and the civil service. The diary provides a unique insight into the political, social, and cultural events of the Restoration period in England, as well as Pepys' personal life. The diary covers a wide range of topics, including the Great Plague of London, the Great Fire of London, the Second Anglo-Dutch War, and Pepys' own


### Descrição contida no dataset

In [22]:
train_dataset[0]['description']

"'the latham-matthews transcription pepys ' diary one glory contemporary english publishing . ' the times 'the pleasure pepys -- reading -- pleasure experience ! pepys ' diary cheerful self-report , man eminent naval history , historical witness , unobjectionable hedonist . ' guardian 'here , one finest feat long history scholarship , pepys ' diary , . exegi monumentum aere perennius . ' observer 'the editor achieved impossible ! one read diary perfectly easily , month month , year year ! last really learned edition learning put disposal layman . ' new statesman 'it n't often one encounter publication -- especially magnitude -- achieves complete perfection , doubt . ' sir arthur bryant"

In [29]:
# a tentativa de treinar com um validation não estava dando um resultado muito legal...

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,  # Certifique-se de que o dataset foi carregado corretamente
    #eval_dataset=eval_dataset,  # Certifique-se de que o dataset foi carregado corretamente
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=4,  # Ajuste o batch size conforme a memória disponível
        gradient_accumulation_steps=8,  # Gradiente acumulado para simular um batch maior
        #num_train_epochs = 3,
        warmup_steps=6,  # Passos de aquecimento da taxa de aprendizado
        max_steps=60,  # Aumente o número de steps se necessário
        learning_rate=2e-4,  # Taxa de aprendizado
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),# Usar BF16 se disponível (para A100)
        logging_steps=1,  # Intervalo para logging
        optim="adamw_8bit",  # AdamW em 8 bits para otimizar memória
        weight_decay=0.01,  # Regularização
        lr_scheduler_type="linear",  # Scheduler linear para a taxa de aprendizado
        seed=3407,  # Semente para reprodutibilidade
        output_dir="outputs",  # Diretório para salvar checkpoints
        #evaluation_strategy="steps",  # Avaliar a cada n steps
        #eval_steps=10,  # Avaliar a cada 5 steps
    ),
)

Map (num_proc=2):   0%|          | 0/76 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [30]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 76 | Num Epochs = 30
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 8
\        /    Total batch size = 32 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,5.314200
2,5.124700
3,4.956500
4,4.737800
5,5.404300
6,4.291200
7,3.916900
8,3.798300
9,3.422200
10,3.284500


In [42]:
#{
#    "instruction": "Summarize this content",
#    "input": "Girls Ballet Tutu Neon Pink",
#    "output": "High quality 3 layer ballet tutu. 12 inches in length"
#}

# Executando pergunta antes do treino
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        train_dataset[62]['title'], # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<s> ### Instruction:
Generate a comprehensive description based on the provided title.

### Title:
wall image offering vietnam veteran memorial

### Description:
volume 1 of 2 the vietnam veteran memorial fund is proud to present this book containing the names of the men and women who died in service in the vietnam war 1955-1975 the vvmf is the nonprofit organization that built the vietnam veteran memorial in washington d.c. and continues to honor and remember the service of our veterans through numerous programs and services this book is a tribute to the more than 58,000 men and women who made the ultimate sacrifice for their country and to their families and loved ones who continue to remember and honor


In [41]:
train_dataset[62]['description']

'volume published fifth anniversary dedication national shrine includes photograph freelance photographer lope 16 photojournalist excerpt letter placed wailing wall family friend war buddy dead missing whose name engraved therethe mystery death writ stone 58132 time american family album deeply moving simplicity damn brotherwhy didnt come back still love id love came back one piece arm leg mind gone among 75 poignant color blackandwhite picture photographed respect child lifted veteran plant kiss name flower teddy bear flag uniform grace polished black granite introduction vietnam veteran freelance writer norman discus controversy wall design mourns friend substituted patrol may died country god nothing long time thought died 150000 first printing bomc featured selectioncopyright 1987 reed business information inc'

In [29]:
lora_model_dir = "lora_model"

In [30]:
model.save_pretrained(base_location+"trained_model/"+lora_model_dir) # Local saving
tokenizer.save_pretrained(base_location+"trained_model/"+lora_model_dir)


('/content/drive/MyDrive/IA para devs - fiap/Fase 3/Tech Challenge/20240923/trained_model/lora_model-2/tokenizer_config.json',
 '/content/drive/MyDrive/IA para devs - fiap/Fase 3/Tech Challenge/20240923/trained_model/lora_model-2/special_tokens_map.json',
 '/content/drive/MyDrive/IA para devs - fiap/Fase 3/Tech Challenge/20240923/trained_model/lora_model-2/tokenizer.model',
 '/content/drive/MyDrive/IA para devs - fiap/Fase 3/Tech Challenge/20240923/trained_model/lora_model-2/added_tokens.json')

In [14]:


del model
del tokenizer
#del inputs
#del trainer
#del trainer_stats


NameError: name 'model' is not defined

In [28]:
import gc

# Limpar o cache da GPU
torch.cuda.empty_cache()

# Coletar lixo (garbage collection)
gc.collect()

65

In [27]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = base_location+"trained_model/"+lora_model_dir, # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)



NameError: name 'lora_model_dir' is not defined

In [31]:
#{
#    "instruction": "Summarize this content",
#    "input": "Girls Ballet Tutu Neon Pink",
#    "output": "High quality 3 layer ballet tutu. 12 inches in length"
#}

# Executando pergunta antes do treino
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        train_dataset[0]['title'], # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<s>### Instruction:
Generate a detailed description based on the provided title.

### Title:
Facing East: A Pilgrim's Journey into the Mysteries of Orthodoxy

### Description:
In this book, the author, a former Anglican priest, describes his journey from the Anglican Church to the Orthodox Church. He describes his experiences in the Orthodox Church, including the liturgy, the icons, the fasting, and the spirituality. He also describes the differences between the Orthodox Church and the Anglican Church. He also describes his experiences in the Orthodox Church, including the liturgy, the icons, the fasting, and the spirituality. He also describes the differences between the Orthodox Church and the Anglican Church.</s>


## Retreinando um modelo já processado anteriormente

In [32]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,  # Certifique-se de que o dataset foi carregado corretamente
    eval_dataset=eval_dataset,  # Certifique-se de que o dataset foi carregado corretamente
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,  # Ajuste o batch size conforme a memória disponível
        gradient_accumulation_steps=4,  # Gradiente acumulado para simular um batch maior
        warmup_steps=10,  # Passos de aquecimento da taxa de aprendizado
        max_steps=100,  # Aumente o número de steps se necessário
        learning_rate=5e-3,  # Taxa de aprendizado
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),# Usar BF16 se disponível (para A100)
        logging_steps=1,  # Intervalo para logging
        optim="adamw_8bit",  # AdamW em 8 bits para otimizar memória
        weight_decay=0.01,  # Regularização
        lr_scheduler_type="linear",  # Scheduler linear para a taxa de aprendizado
        seed=3407,  # Semente para reprodutibilidade
        output_dir="outputs",  # Diretório para salvar checkpoints
        evaluation_strategy="steps",  # Avaliar a cada n steps
        eval_steps=10,  # Avaliar a cada 5 steps
    ),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/5956 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/1490 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [33]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 5,956 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss,Validation Loss
10,10.092300,9.333007
20,8.771600,8.628121
30,7.589700,7.632223
40,7.774000,7.777703
50,7.679500,7.745951


KeyboardInterrupt: 